In [1]:
import os
import sys
import random
import warnings
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from tqdm import tqdm
from itertools import chain

from keras.utils import to_categorical
from keras.callbacks import Callback

from keras import layers
from keras import optimizers
from keras.models import Model, load_model
from keras.layers import Input, Activation, Dense, Flatten
from keras.layers import Concatenate, multiply
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Conv1D
from keras.layers import ZeroPadding1D
from keras.layers import GlobalAveragePooling1D
from keras.layers.pooling import MaxPooling1D

from keras.layers import Deconv2D,LeakyReLU,add,Reshape
from keras import losses

from keras import regularizers
from keras.layers import Conv2D

from keras.losses import categorical_crossentropy

from keras.layers.core import Lambda

from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score

from keras.models import model_from_json

from keras import  backend as K

import tensorflow as tf



os.environ["CUDA_VISIBLE_DEVICES"] = "0"


import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
session = tf.Session(config=config)

KTF.set_session(session )

Using TensorFlow backend.
/home/c1501f/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/c1501f/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/c1501f/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c1

In [2]:
train_batch = 128
test_batch = 128

train_num = 40020
test_num = 13108

cross = "cross_1"

snr_str = "-6_db"

In [3]:
def generate_train_data():
    
    train_data = np.zeros((train_batch, 2048, 1), dtype = np.float32)
    d_train_lab = np.zeros((train_batch, 2048, 1), dtype = np.float32)
    c_train_lab = np.zeros((train_batch))
    
    flag = 0
    
    while True:
        
        list = random.sample(range(train_num), train_num)
        
        for id in list:
            
            num_id = str(id)
            
            np_train_data = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/train_data/" + num_id + "_train.npy")
            npd_train_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/no_db/train_data/" + num_id + "_train.npy") 
            npc_train_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/train_lab/" + num_id + "_lab.npy")
            
            train_data[flag, :, :] = np_train_data
            d_train_lab[flag, :, :] = npd_train_lab    
            c_train_lab[flag] = npc_train_lab
            
            flag += 1
            
            if flag >= train_batch:
                
                train_hot_lab = to_categorical(c_train_lab, num_classes=10)
                
                yield [train_data], [train_hot_lab]
                
                flag = 0
                train_data = np.zeros((train_batch, 2048, 1), dtype = np.float32)
                d_train_lab = np.zeros((train_batch, 2048, 1), dtype = np.float32)
                c_train_lab = np.zeros((train_batch))

        
def generate_test_data():
    
    test_data = np.zeros((test_batch, 2048, 1), dtype = np.float32)
    d_test_lab = np.zeros((test_batch, 2048, 1), dtype = np.float32)
    c_test_lab = np.zeros((test_batch))
    
    flag = 0
    
    while True:
        
        list = random.sample(range(test_num), test_num)
        
        for id in list:
            
            num_id = str(id)
            
            np_test_data = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/test_data/" + num_id + "_test.npy")
            npd_test_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/no_db/test_data/" + num_id + "_test.npy") 
            npc_test_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/test_lab/" + num_id + "_lab.npy")
            
            test_data[flag, :, :] = np_test_data
            d_test_lab[flag, :, :] = npd_test_lab    
            c_test_lab[flag] = npc_test_lab
            
            flag += 1
            
            if flag >= test_batch:
                
                test_hot_lab = to_categorical(c_test_lab, num_classes=10)
                
                yield [test_data], [test_hot_lab]
                
                flag = 0
                test_data = np.zeros((test_batch, 2048, 1), dtype = np.float32)
                d_test_lab = np.zeros((test_batch, 2048, 1), dtype = np.float32)
                c_test_lab = np.zeros((test_batch))

In [4]:
def get_all_test_data():
    
    test_data = np.zeros((test_num, 2048, 1), dtype = np.float32)
    d_test_lab = np.zeros((test_num, 2048, 1), dtype = np.float32)
    c_test_lab = np.zeros((test_num))
    
    flag = 0
    list = random.sample(range(test_num), test_num)
    
    for id in list:
        
        num_id = str(id)
        
        np_test_data = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/test_data/" + num_id + "_test.npy")
        npd_test_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/no_db/test_data/" + num_id + "_test.npy") 
        npc_test_lab = np.load("../../../Data/Xi_W_all/denoise_data/"+cross+"/"+snr_str+"/test_lab/" + num_id + "_lab.npy")
            
        test_data[flag, :, :] = np_test_data
        d_test_lab[flag, :, :] = npd_test_lab    
        c_test_lab[flag] = npc_test_lab
        
        flag += 1
        
    test_hot_lab = to_categorical(c_test_lab, num_classes=10)
    
    return test_data, d_test_lab, test_hot_lab

In [5]:
slope = 0.3

def Denosing_CNN2():
    
    input = Input(shape=[2048, 1])

    D1 = Conv1D(filters=16, kernel_size=12, strides=4, padding="same")(input)
    D1 = LeakyReLU(alpha=slope)(D1)
        
    D2 = Conv1D(filters=32, kernel_size=6, strides=4, padding="same")(D1)
    D2 = LeakyReLU(alpha=slope)(D2)    
    
    D3 = Conv1D(filters=64, kernel_size=3, strides=2, padding="same")(D2)
    D3 = LeakyReLU(alpha=slope)(D3)
    
    D4 = Conv1D(filters=128, kernel_size=3, strides=2, padding="same")(D3)
    D4 = LeakyReLU(alpha=slope)(D4)    
      

    Fx_5 = Conv1D(256, kernel_size = 3, strides=2, padding='same')(D4)
    Fx_5 = LeakyReLU(alpha=slope)(Fx_5)   
    
    Fx_6 = GlobalAveragePooling1D()(Fx_5)
    Fx_6 = Dense(10,  activation='softmax', name='out2')(Fx_6)     
 
         
    Total_Model = Model(inputs=input, outputs= Fx_6)  
    
    return Total_Model

In [6]:
adam = optimizers.adam(lr = 0.0001)

model = Denosing_CNN2()

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2048, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 16)           208       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 32)           3104      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128, 32)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 64, 64)            6208      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 64, 64)            0         
__________

In [ ]:
class MyCbk(Callback):

    def __init__(self, model):
        self.model_to_save = model
            
    def on_epoch_end(self, epoch, logs=None):
        
        print('save model----model_at_epoch_%d.h5' % epoch)
        self.model_to_save.save('model/8/model_%d.h5' % epoch)
        
cbk = MyCbk(model)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.90, patience=5, verbose=0, mode='auto', cooldown=0, min_lr=0.0000001)

results = model.fit_generator(generate_train_data(), steps_per_epoch = train_num/train_batch, epochs = 100, validation_data = generate_test_data(), validation_steps = test_num/test_batch, verbose=1, callbacks=[cbk, reduceLR])

Epoch 1/100


In [ ]:
all_test_data, all_d_lab, all_hot_lab = get_all_test_data()


Accuracy_list = np.zeros((100))

In [ ]:
for model_id in range(99, 100):
    
    str_id = str(model_id)
    model =  load_model('model/8/model_'+ str_id +'.h5') 
    
    range_num = int(test_num/test_batch)
    
    input_hot_lab = []
    output_hot_lab = []   
    
    for num_id in range(range_num):
        
        test_data = all_test_data[num_id*test_batch:test_batch+num_id*test_batch,:,:]
        hot_lab = all_hot_lab[num_id*test_batch:test_batch+num_id*test_batch,:]
    
        pre_hot = model.predict(test_data, batch_size=test_batch, verbose=0)
           
        input_hot_lab.extend(hot_lab)
        output_hot_lab.extend(pre_hot)
        
    hot_labels = np.argmax(input_hot_lab,axis=1)  
    pre_hot_labels = np.argmax(output_hot_lab,axis=1)
          
    Accuracy = accuracy_score(hot_labels, pre_hot_labels)
    Accuracy_list[model_id] = Accuracy
    
    print("第"+ str(model_id) + "个模型的结果：")
    print(Accuracy)
    
    print("------------------------------------")
    
    K.clear_session()
    tf.reset_default_graph()


In [ ]:
Accuracy_str = "["

for id in range(80, 100):
    
    Accuracy_num = round(Accuracy_list[id], 5)

    Accuracy_str = Accuracy_str + str(Accuracy_num)
    
    if id == 99:
        Accuracy_str = Accuracy_str + "]"

    else:
        Accuracy_str = Accuracy_str + ", "

print("Accuracy_score:")
print(Accuracy_str)


print('Accuracy: '+str(Accuracy_list[80:99].max()))

print("**------------**-------------**---------------**")